【数据背景】新能源车电池模组的充放电数据是指电池在使用过程中进行充电和放电时所记录的相关信息。这些数据包含了电池的性能指标、工作状态和使用情况，可以帮助监测和评估电池的健康状况、性能表现以及寿命预测。这些数据通过车辆的电池管理系统（Battery Management System，简称BMS）进行监测和记录。基于这些数据，制造商和维护人员可以进行电池性能分析、故障诊断以及优化电池使用和充电策略。同时，这些数据也可以用于研究和改进电池技术，提高新能源车的续航里程和可靠性。

【应用领域】AI+电池充放电

【文件目录】20个#x的csv数据文件（比如第一辆的名称就是#1.csv）

【数据说明】共20辆新能源车的电池模组充放电数据，每个#x代表一辆新能源车电池模组的充放电数据，数据时间段为2019/07/25—2021/11/15，数据跨度约29个月，数据集大小约1.1GB。每个csv数据表包含10个数据字段，具体如下：

1. record_time：时间戳
2. soc：车辆电池剩余容量，单位为%
3. pack_voltage：电池模组电压，单位为V
4. charge_current：充电电流，单位为A
5. max_cell_voltage：电池单体最大电压，单位为V
6. min_cell_voltage：电池单体最小电压，单位为V
7. max_temperature：最高温度，单位为℃
8. min_temperature：最低温度，单位为℃
9. available_energy：可用能量，单位为kW
10. available_capacity：可用容量，单位为Ah

In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns

# 设置绘图样式  
sns.set(style="whitegrid")

# 1. 导入所有的CSV文件  
data_files = glob.glob("./*.csv")  # 假设CSV文件在data目录下  
dataframes = []
data_files

for file in data_files:
    df = pd.read_csv(file)
    df = df.iloc[:, 1:] # 去掉第一行
    # 追加文件名作为列以便之后区分不同车辆
    df['vehicle_id'] = file.split('\\')[-1].split(".")[0]  # 获取文件名作为车辆识别
    dataframes.append(df)

# 合并所有数据框
data = pd.concat(dataframes, ignore_index=True)

# - 将record_time转换为datetime格式  
data['record_time'] = pd.to_datetime(data['record_time'], format='%Y%m%d%H%M%S', errors='coerce')  

# - 删除缺失值  
data.dropna(inplace=True)

,record_time,soc,pack_voltage (V),charge_current (A),max_cell_voltage (V),min_cell_voltage (V),max_temperature (℃),min_temperature (℃),available_energy (kw),available_capacity (Ah),vehicle_id
0,20190726200235,27.2,328.2,-52.20001,3.656,3.640,41,38,12.40,37.28,#1
1,20190726200243,27.6,328.5,-52.20001,3.663,3.645,41,38,12.44,37.39,#1
2,20190726200251,27.6,328.6,-52.20001,3.665,3.647,41,38,12.47,37.51,#1
3,20190726200259,27.6,328.6,-52.20001,3.666,3.649,41,38,12.52,37.64,#1
4,20190726200307,27.6,328.8,-52.20001,3.666,3.649,41,38,12.56,37.76,#1
...,...,...,...,...,...,...,...,...,...,...,...
16100723,20211115191833,74.4,356.4,-69.89999,3.986,3.947,33,31,32.50,97.72,#9
16100724,20211115191843,74.8,356.6,-69.80002,3.986,3.949,33,31,32.57,97.95,#9
16100725,20211115191853,74.8,356.8,-69.89999,3.988,3.950,33,31,32.65,98.18,#9
16100726,20211115191903,75.2,356.9,-70.00000,3.990,3.952,33,31,32.73,98.41,#9


In [ ]:
# 划分训练集和测试集  
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor  
from sklearn.metrics import mean_squared_error, r2_score

# 设定目标变量 y，可以根据具体需要进行自定义  
data['health_status'] = data['soc']  # 使用剩余容量作为健康状况的简化表示  
# 可以进一步细分为是否健康等标签

# 选择特征  
features = ['pack_voltage (V)', 'charge_current (A)',   
            'max_cell_voltage (V)', 'min_cell_voltage (V)',   
            'max_temperature (℃)', 'min_temperature (℃)',   
            'available_energy (kw)', 'available_capacity (Ah)']  
X = data[features]
y = data['health_status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 初始化随机森林模型  
n_estimators = 100  # 树的数量
model = RandomForestRegressor(n_estimators=n_estimators, random_state=42)

for i in range(n_estimators):
    model.estimators_ = []  # 重置
    model.n_estimators = i + 1  # 增加树的数量
    model.fit(X_train, y_train)  # 训练当前树
    print(f'Trained {i + 1}/{n_estimators} trees')

# 预测  
y_pred = model.predict(X_test)

# 评估模型性能  
mse = mean_squared_error(y_test, y_pred)  
r2 = r2_score(y_test, y_pred)  

print(f'Mean Squared Error: {mse}')  
print(f'R^2 Score: {r2}')

import matplotlib.pyplot as plt  

plt.scatter(y_test, y_pred)  
plt.xlabel('Actual Health Status')  
plt.ylabel('Predicted Health Status')  
plt.title('Actual vs Predicted Health Status')  
plt.show()  